01 特徴量ハッシング
================

* `ハッシュ関数`：潜在的に無限に広がる値を、有限の$m$種類の値に割り当てる関数

    * ここでは、以下の通りに定義する
    
        *  `ハッシュ値`：ハッシュ関数を適用した後に得られる値
        
        * `ハッシュテーブル`：ハッシュ値を格納するテーブル
        
* ハッシュテーブルのサイズは、$m$となる

* 入力値の取りうる範囲は出力値の取りうる範囲よりも広いため、値が異なる入力値が同じ出力値に割り当てられることがある

    * これを、`衝突`と呼ぶ
    
* `一様なハッシュ関数`は、ハッシュ値が$m$種類のそれぞれにおおよそ同じ回数だけ割り当てられることを保証する

* イメージとしては、ハッシュ関数は、番号付きボール(キー)を取り込んで$m$個のビン(容器)の1つに割り当てるマシンと捉えることができる

    * 同じ番号のボールは、常に同じビンに割り当てられる
    
    * このような特性により、特徴量の値を有限な範囲内に維持しつつ、機械学習の学習と評価のサイクルに必要となる記憶容量と計算時間を減らす

![ハッシュ関数によってキーをビンに割り当てる](./images/ハッシュ関数によってキーをビンに割り当てる.png)

* ハッシュ関数は、数値で置き換えられる任意のオブジェクト(コンピュータに格納できるデータ)に適用できる
    
    * 例)数値、文字列、複雑な構造データなど

* 非常に多くの特徴量があるとき、特徴ベクトルを保存するには多くの容量が必要となる

    * `特徴量ハッシング`は、特徴量IDにハッシュ関数を適用することで、特徴ベクトルを$m$次元のベクトルに圧縮する
    
    * 例)特徴量が文書内の単語の場合、大量の種類の単語が含まれていても、特徴量ハッシングによって$m$個の特徴量に圧縮できる

In [1]:
# 単語特徴量のための特徴量ハッシング
def hash_features(word_list, m):
    output = [0] * m
    for word in word_list:
        index = hash_fcn(word) % m
        output[index] += 1
    return output

* 特徴量ハッシングの別のバリエーションは、符号コンポーネントを持っており、ハッシュ値のカウントの加算や減算ができる

    * この符号コンポーネントを持つ特徴量ハッシングを適用する前後では、特徴ベクトル間の内積の期待値は変わらないことが知られている
    
    * つまり、特徴量ハッシングによって大きなバイアスが発生することはない

In [2]:
# 符号化特徴量ハッシング
def hash_features(word_list, m):
    output = [0] * m
    for word in word_list:
        index = hash_fcn(word) % m
        sign_bit = sign_hash(word) % 2
        if (sign_bit == 0):
            output[index] -= 1
        else:
            output[index] += 1
    return output

* 特徴量ハッシング後の内積の値は、元の内積の$O$($\frac{1}{\sqrt{m}}$)の範囲内にあることが知られているので、そこから誤差を計算できる

    * そして、許容可能な誤差に基づいてハッシュテーブルのサイズ$m$を選択できる
    
    * ただ、実際には$m$の値は試行錯誤して決定することが多い

* 特徴量ハッシングは、線形モデルやカーネル法など、特徴ベクトルと係数ベクトルの内積を含むモデルに利用できる

    * 応用例)迷惑メールフィルタリングにおいて有効
    
    * 一方で、ターゲティング広告の場合に、予測誤差を許容可能な範囲に収めるためには$m$を数十億のオーダーにする必要がある(容量の節約にならない)

* 特徴量ハッシングの欠点は、特徴量ハッシング後の特徴量が解釈が難しくなる点

* 実際に、Yelpレビューデータセットを例に、scikit-learnの`FeatureHasher`クラスを適用した際の、解釈可能性と記憶容量のトレードオフの関係を示す

In [3]:
import pandas as pd
import json

# 最初の10,000件のレビューを読み込み
with open('/Users/kunii.sotaro/Downloads/review.json') as f:
    js = []
    for i in range(10000):
        js.append(json.loads(f.readline()))

review_df = pd.DataFrame(js)
# mにbusiness_idのユニーク数を代入
m = len(review_df['business_id'].unique())

m

4618

In [4]:
from sklearn.feature_extraction import FeatureHasher
h = FeatureHasher(n_features=m, input_type='string')
f = h.transform(review_df['business_id'])

# 変換後の特徴量が解釈が困難であることを確認
review_df['business_id'].unique().tolist()[0:5]

['ujmEBvifdJM6h6RLv4wQIg',
 'NZnhc2sEQy3RmzKTZnqtwQ',
 'WTqjgwHlXbSFevF32_DJVw',
 'ikCg8xy5JIg_NGPx-MSIDA',
 'b1b1eb3uo-w561D0ZfCEiQ']

In [5]:
f.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [6]:
# 変換後の特徴量のストレージサイズが大きく減っていることを確認
from sys import getsizeof
print('Our pandas Series, in bytes: ', getsizeof(review_df['business_id']))
print('Our hashed numpy array, in bytes: ', getsizeof(f))

Our pandas Series, in bytes:  790104
Our hashed numpy array, in bytes:  56


* 特徴量ハッシングを適用すると、容量が小さくなり計算が容易になる反面、直感的な解釈が難しくなることがわかる

* 解釈を目的としているデータの探索や視覚化においては、特徴量ハッシングを適用することは好ましくない

    * 一方、大規模なデータセットを利用した機械学習においては、直感的な解釈の重要性は低く、特徴量ハッシングの適用するメリットを十分に享受できる

| 版   | 年/月/日   |
| ---- | ---------- |
| 初版 | 2019/05/04 |